# ADK 101 in 45min


Where to test:
- stdout output
- `adk web` as UI -- very nice visualizations

Pre-caution:
- ADK is still under development. The official launch is early April. This training is intended to provide hands-on experience with the Agent Development Kit (ADK) and its foundational concepts.


Documents 
https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk



# Environment


In [1]:
# !pip install -r requirements.txt

In [2]:
# !pip3 install google_adk 
# !pip install -q --upgrade google-cloud-discoveryengine
# !pip install google-adk
# !pip install google-genai
# !pip install pydantic
!pip install --upgrade opentelemetry-sdk opentelemetry-api "google-cloud-aiplatform[all]"
#-0.0.2.dev20250324+739344859-py3-none-any.whl

  Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.55b1-py3-none-any.whl.metadata (2.5 kB)
Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl (118 kB)
Using cached opentelemetry_api-1.34.1-py3-none-any.whl (65 kB)
Using cached opentelemetry_semantic_conventions-0.55b1-py3-none-any.whl (196 kB)
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.33.1
    Uninstalling opentelemetry-api-1.33.1:
      Successfully uninstalled opentelemetry-api-1.33.1
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.54b1
    Uninstalling opentelemetry-semantic-conventions-0.54b1:
      Successfully uninstalled opentelemetry-semantic-conventions-0.54b1
  Attempting uninstall: opentelemetry-sdk
    Found existing installation: opentelem

### Restart current runtime
To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.
<div class="alert alert-block alert-success"> 
<b>NOTE:</b> Only restart the current runtime if you installed libraries. If you did not install new libraries, you do not need to restart the kernel.
</div>

In [3]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython
# import time

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Import neccessary libraries

In [4]:
import os

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

# Print project and location details
print(f"Project ID:", PROJECT_ID)
print(f"Project Region:", LOCATION)

Project ID: my-project-0004-346516
Project Region: us-central1


In [5]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = LOCATION
UNIQUE_PREFIX = 'sm' #"<REPLACE_WITH_PREFIX>"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

else:    
    print(f"\n{GCS_BUCKET_NAME} already exists.")
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)



my-project-0004-346516-sm already exists.


In [6]:
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

LOCATION = "us-central1" #@param {type:"string"}
STAGING_BUCKET = GCS_BUCKET_URI

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [7]:
# !gcloud storage ls gs://adk_training/sdk
# !gcloud storage cp gs://adk_training/sdk/google_adk-0.0.2.dev20250324+739344859-py3-none-any.whl .

In [8]:
import random

from google.adk.runners import InMemoryRunner, Runner
from google.adk.agents import BaseAgent, LlmAgent, Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.tools import ToolContext, LongRunningFunctionTool
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import Session
from google.adk.events import Event
from google.adk.agents.callback_context import CallbackContext

from google.genai import types

from pydantic import BaseModel, Field
from typing import List, Optional
import logging

logging.basicConfig(level=logging.ERROR)

# Utils

In [9]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()

    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()

        try:
            agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))
        except AttributeError as e:
            print(f"Error parsing event content: {e}")
            continue

        if is_final_response:
            final_response = event.content.parts[0].text if event.content.parts else "No content available"
            elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
            print(f'>>> Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-" * 30)
        elif function_calls:
            print('+++ Function Calls:')
            for function_call in function_calls:
                print(f"Function Name: {function_call.name}, Args: {function_call.args}")
        elif function_responses:
            print('--- Function Responses:')
            for function_response in function_responses:
                response_details = function_response.response
                recommended_list = list(response_details.values()) if response_details else []
                print(f"Function Name: {function_response.name}")
                print(f"Function Results: {json.dumps(recommended_list)}")
        else:
            print('No function calls or responses available.')
            print(f"Agent Response: {agent_res}")

    elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
    print(f"Total elapsed time: {elapsed_time_ms} ms")


In [10]:
from typing import Optional, Any

APP_NAME = 'test_app'
USER_ID = 'test_user'

def create_runner(agent: BaseAgent):
    return InMemoryRunner(agent, app_name=APP_NAME)

def _content_to_text(content: types.Content | None) -> str:
    if not content or not content.parts:
        return ''
    return ''.join([p.text or '' for p in content.parts])

def print_event(event: Event):
    print(f'Author: {event.author}')
    print(f'Content Text: {_content_to_text(event.content)}')
    print(f'Event: {event.model_dump(exclude_none=True, exclude_defaults=True)}')

async def run_session(new_message: types.Content, *, runner: Runner, session: Optional[Session] = None, state: Optional[dict[str, Any]] = None) -> Session:
    if session is None:
        session = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, state=state)
        
    print(f'User: {_content_to_text(new_message)}')
    print('----------------------------------')
    async for e in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=new_message):
        print_event(e)
    print('----------------------------------')
    
    session = runner.session_service.get_session(app_name=session.app_name, user_id=session.user_id, session_id=session.id)
    return session

def content_text(msg: str) -> types.Content:
    return types.UserContent(parts=[types.Part(text=msg)])

# Hello world!

Concepts:

- Agent - Starts as a wrapper of LLM that can decide actions and interact with external world (other agents, tools)
- Session - Maintains the state of one invocation. (chat log / events/ etc)
- Runner - The orchestration layer of agents. The "state machine". Runner drives agents to move the states of a session forward.

Use case:
- An agent that says hello word in a random language

In [11]:
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.tools.agent_tool import AgentTool
from google.adk.sessions import InMemorySessionService

import warnings
import logging
from pprint import pprint

logging.getLogger('google_genai.types').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [12]:
# Constant
APP_NAME = "hello_world_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_word_agent"
MODEL = "gemini-2.0-flash-001"

**Instructions:** Start by defining what your agent should do using simple natural language instructions. This provides a very straightforward way to customize agent actions without needing to compose complex chains or graphs. You can even reference context like user information and current time within these instructions, which will be filled in at runtime




**Sessions:** The Agent Framework handles session management for you, allowing you to focus on building your agent's logic. You can use the provided BaseSession interface with managed storage or easily plug in your own. The InMemorySession is available for rapid development and debugging

In [13]:
# Agent
hello_world_agent = Agent(
    model=MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)

    Example Output 2:
    你好，世界 (Chinese)
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_world_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    return events

In [14]:
events = call_agent(runner, "hello")
pprint_events(events)

Exception in thread Thread-5 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 143, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/opt/conda/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.10/site-packages/google/adk/runners.py", line 131, in _invoke_run_async
    async for event in self.run_async(
  File "/opt/conda/lib/python3.10/site-packages/google/adk/ru

Total elapsed time: 5.273 ms


In [15]:
session

<coroutine object InMemorySessionService.create_session at 0x7faae4939e00>

In [16]:
# session.id

In [17]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

#TypeError: InMemorySessionService.list_sessions() missing 2 required keyword-only arguments: 'app_name' and 'user_id'


<coroutine object InMemorySessionService.list_sessions at 0x7faae493a490>

In [18]:
# pprint(session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events)

# Multi turn conversation

Concepts:

- Event(s)
- Session
- SessionService: sessions[app][user][session]

Use case:

- An agent that tries to know your name and send hello USERNAME

In [19]:
import asyncio
from google.genai import types
# from google.adk import Agent, Runner, InMemorySessionService

# Constants
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"
MODEL = "gemini-1.5-flash"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name=AGENT_NAME,
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.
    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

async def main():
    # Session and Runner
    session_service = InMemorySessionService()
    session = await session_service.create_session(
        app_name=APP_NAME, 
        user_id=USER_ID, 
        session_id=SESSION_ID
    )
    runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)
    
    # Agent Interaction - NO AWAIT HERE, runner.run() is synchronous
    def call_agent(runner, session, query):
        content = types.Content(role='user', parts=[types.Part(text=query)])
        events = runner.run(  # NO AWAIT HERE
            user_id=session.user_id, 
            session_id=session.id, 
            new_message=content
        )
        return events
    
    events = call_agent(runner, session, "hello")  # NO AWAIT HERE
    
    # Convert generator to list to see all events
    events_list = list(events)
    print("Events:", events_list)
    
    return events_list

# Run the async function
events = await main()

Events: [Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Hello there!  Before we begin, could I please ask for your name?  It doesn't have to be your full name – a first name, nickname, or whatever you're comfortable sharing will do.  Knowing your name will help me personalize our interaction.\n")], role='model'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=56, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=56)], prompt_token_count=93, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=93)], thoug

In [20]:
import asyncio
from google.genai import types
# from google.genai.adk import Agent, Runner, InMemorySessionService

# Constants
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"
MODEL = "gemini-1.5-flash"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name=AGENT_NAME,
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.
    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Setup everything
session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)
runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)

# Agent interaction function
def call_agent(runner, session, query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(
        user_id=session.user_id, 
        session_id=session.id, 
        new_message=content
    )
    return list(events)  # Convert generator to list immediately

# Test the agent
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> Final Response (0.046 ms):
I understand you may be hesitant to share your name, but it would help me to personalize our interaction.  Even a nickname will do!  Knowing your name, even just a part of it, allows me to address you more appropriately and makes our conversation feel a bit more friendly.  Would you be willing to share at least a portion of your name with me?

------------------------------
Total elapsed time: 0.173 ms


In [21]:
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> Final Response (0.049 ms):
Okay, I respect your decision not to share your name.  Let's proceed without it.

------------------------------
Total elapsed time: 0.173 ms


In [22]:
events = call_agent(runner, session, "what is your name?")
pprint_events(events)

>>> Final Response (0.045 ms):
My internal name is hello_name_agent.  My description is "An agent that says 'hello USERNAME'".

------------------------------
Total elapsed time: 0.146 ms


In [23]:
events = call_agent(runner, session, "my name is Amir")
pprint_events(events)

>>> Final Response (0.044 ms):
Hello Amir!

------------------------------
Total elapsed time: 0.165 ms


In [24]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (0.043 ms):
Your name is Amir.

------------------------------
Total elapsed time: 0.146 ms


## Switch session

In [25]:
new_session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID + '-new')

In [26]:
events = call_agent(runner, new_session, "what is my name?")
pprint_events(events)

>>> Final Response (0.047 ms):
Hello!  I need your name to properly greet you.  It can be your first name, last name, nickname – whatever you'd like me to call you.  Could you please tell me your name?

------------------------------
Total elapsed time: 0.16 ms


In [27]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> Final Response (0.068 ms):
Your name is Amir.

------------------------------
Total elapsed time: 0.189 ms


# Use Tool

**Function Tools:** To extend your agent's capabilities, you can provide it with Python functions as tools. When the agent determines it needs to perform a specific action, it can automatically call these functions

Concept:
- Tool (Python function) -- the experience of tool calling by agent (by LLM) or by human (from Python) are the same. Easy to test.

Use case:
- Simple math

In [28]:
def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result


In [29]:
add([2, 2, 3])

7

In [30]:
multiply([2, 2, 3])

12

In [31]:
simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

In [33]:
import random

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
    """create a pre-configured agent caller.
    Args:
        root_agent: The ADK agent to handle conversations
        app_name: Application name (default: 'App12345')
        user_id: User identifier (default: 'User12345')
        session_id: Optional session ID. If None, generates a random one.
    Returns:
        A function that takes a query string and returns agent response events.
    """
    session_service = InMemorySessionService()
    if session_id is None:
        suffix = random.randint(100000, 999999)
        session_id = f'{app_name}-{user_id}-{suffix}'
    
    session = await session_service.create_session(
        app_name=app_name, 
        user_id=user_id, 
        session_id=session_id
    )
    runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
    
    def _call(query):
        content = types.Content(role='user', parts=[types.Part(text=query)])
        events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
        return list(events)  # Convert generator to list
    
    return _call

In [36]:
# Correct way - use await to get the actual callable function
call = await caller_factory(root_agent=simple_math_agent)
pprint_events(call('hello'))

/var/tmp/ipykernel_1219651/3527633880.py:2: RuntimeWarning: coroutine 'caller_factory' was never awaited
  call = await caller_factory(root_agent=simple_math_agent)


>>> Final Response (0.065 ms):
Hello!  I can perform addition, subtraction, multiplication, and division operations on numbers you provide.  Tell me the numbers you want to operate on. For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'. You can also provide a range: 'Multiply the numbers between 1 and 10'.

------------------------------
Total elapsed time: 0.18 ms


In [37]:
pprint_events(call('what is three plus 9?'))

+++ Function Calls:
Function Name: add, Args: {'numbers': [3, 9]}
--- Function Responses:
Function Name: add
Function Results: [12]
>>> Final Response (0.27 ms):
12

------------------------------
Total elapsed time: 0.291 ms


In [38]:
pprint_events(call('multiply that by 2'))

+++ Function Calls:
Function Name: multiply, Args: {'numbers': [12, 2]}
--- Function Responses:
Function Name: multiply
Function Results: [24]
>>> Final Response (0.251 ms):
24

------------------------------
Total elapsed time: 0.272 ms


In [41]:
call = await caller_factory(root_agent=simple_math_agent)
pprint_events(call('Here is my math problem that is about apple counting. Let us start saying that I have three apple'))

/var/tmp/ipykernel_1219651/3373569892.py:1: RuntimeWarning: coroutine 'caller_factory' was never awaited
  call = await caller_factory(root_agent=simple_math_agent)


>>> Final Response (0.053 ms):
Okay, you have three apples. What would you like to do with them?  I can add, subtract, multiply, or divide the number of apples.  Tell me your next instruction.

------------------------------
Total elapsed time: 0.163 ms


In [42]:
pprint_events(call('Alice gave anoter 2 apples'))

>>> Final Response (0.051 ms):
Okay, Alice gave you 2 more apples.  So you started with 3 apples and received 2 more.  That means you now have 3 + 2 = 5 apples.

------------------------------
Total elapsed time: 0.168 ms


In [43]:
pprint_events(call('how many apples do I have?'))

>>> Final Response (0.067 ms):
You have 5 apples.

------------------------------
Total elapsed time: 0.178 ms


In [44]:
pprint_events(call('Bob gave me 3 apples. how many apples do I have?'))

>>> Final Response (0.055 ms):
You had 5 apples, and Bob gave you 3 more.  Therefore, you now have 5 + 3 = 8 apples.

------------------------------
Total elapsed time: 0.17 ms


In [45]:
pprint_events(call('Alice and Bob do this to me for three more times. how many apples do I have?'))

>>> Final Response (0.082 ms):
Each time, you receive a total of 2 + 3 = 5 apples from Alice and Bob. This happens three more times.  So you'll receive an additional 5 * 3 = 15 apples.

You started with 8 apples, and you'll receive 15 more, giving you a total of 8 + 15 = 23 apples.

------------------------------
Total elapsed time: 0.216 ms


In [46]:
pprint_events(call('yes your logic is correct'))

>>> Final Response (0.069 ms):
Great! Is there anything else I can help you calculate?

------------------------------
Total elapsed time: 0.212 ms


# Agent As Tool

**AgentTool:** You can embed the power of one agent within another by using AgentTool. This allows you to treat an entire agent as a tool within a parent agent. AgentTools are executed in an isolated environment, promoting safety and modularity. Combined with input/output schemas, this enables the creation of sophisticated interactions


Function Tool vs Agent Tool:
 If the execution flow goes from Agent to Tool and it always come back to theinitiation agent vs if the execution flow goes from Agent to **another** agent, the latter agent can decide where the flow goes next.

Concepts:
- AgentTool -- If you want another agent to handle a task and always come back to the caller, make this `Agent` a `Tool` by `AgentTool`

Use case:
- Advanced math agent

In [47]:
agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [48]:
call = await caller_factory(root_agent=agent_math_advanced)

In [49]:
pprint_events(call('who are you?'))

>>> Final Response (0.08 ms):
I am agent_math_advanced.  The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.

------------------------------
Total elapsed time: 0.258 ms


In [50]:
pprint_events(call('what is 1 + (three+2) times 7 '))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The result is 5.\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1 + 5 * 7'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["Following the order of operations (PEMDAS/BODMAS), multiplication is performed before addition.\n\n1. **Multiplication:** 5 * 7 = 35\n2. **Addition:** 1 + 35 = 36\n\nTherefore, the answer is $\\boxed{36}$\n"]
>>> Final Response (0.342 ms):
The result of "1 + (three+2) times 7" is 36

------------------------------
Total elapsed time: 0.363 ms


In [51]:
call = await caller_factory(root_agent=agent_math_advanced)

In [52]:
pprint_events(call('add from 1 to 2. then multiply that by 2.'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '1+2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["3\n"]
+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '3*2'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["The answer is 6.\n"]
>>> Final Response (0.344 ms):
The result of "((1+2)*2)" is 6.

------------------------------
Total elapsed time: 0.364 ms


In [53]:
call = await caller_factory(root_agent=agent_math_advanced)
pprint_events(call('how much is 10 / 5 + three times 5?'))

+++ Function Calls:
Function Name: simple_math_agent, Args: {'request': '10 / 5 + 3 * 5'}
--- Function Responses:
Function Name: simple_math_agent
Function Results: ["Following the order of operations (PEMDAS/BODMAS), we calculate:\n\n1. **Division:** 10 / 5 = 2\n2. **Multiplication:** 3 * 5 = 15\n3. **Addition:** 2 + 15 = 17\n\nTherefore, the answer is $\\boxed{17}$\n"]
>>> Final Response (0.346 ms):
The result is 17.

------------------------------
Total elapsed time: 0.384 ms


**NOTE**: This part is very nice, it shows how two agents interact to clarify the input type should be integers and input again.

```
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I can only work with integers. Could you provide the numbers as integers?\n"]
WARNING:google_genai.types:Warning: there are non-text parts in the response: ['function_call'],returning concatenated text from text parts,check out the non text parts for full response from model.
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}

```

# Input/Output Format Control

**Input Schema / Output Schema:** Ensure data consistency and validity by defining Pydantic schemas for your agent's input and output. The agent can then verify the input against the schema and guarantee that the generated output conforms to the specified structure using constrained decoding

In [54]:
from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
json_schema

{'properties': {'original_query': {'description': 'The original text from user.',
   'title': 'Original Query',
   'type': 'string'},
  'corrected_text': {'description': 'The corrected text.',
   'title': 'Corrected Text',
   'type': 'string'},
  'errors': {'description': 'An array of descriptions of each error.',
   'items': {'type': 'string'},
   'title': 'Errors',
   'type': 'array'},
  'explanations': {'description': 'An array of explanations for each correction.',
   'items': {'type': 'string'},
   'title': 'Explanations',
   'type': 'array'}},
 'required': ['original_query', 'corrected_text', 'errors', 'explanations'],
 'title': 'OutputSchema',
 'type': 'object'}

In [55]:
# 1. The {json_schema} in instruction is the key for model to follow the schema.
# 2. The output_schema=OutputSchema provides a validation step after model output.

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
    disallow_transfer_to_parent = True,
    disallow_transfer_to_peers=True
    
    # allow_transfer=False,
)

In [56]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=agent_grammar,
    enable_tracing=True,
)

In [59]:
for event in app.stream_query(
    user_id="u_123",
    # session_id=session.id,
    message="whats the weather in new york",
):
    print(event)

{'content': {'parts': [{'text': '{"original_query": "whats the weather in new york", "corrected_text": "What is the weather in New York?", "errors": ["The sentence starts with a lowercase letter and uses informal contractions.", "New York should be capitalized."], "explanations": ["Sentences always start with a capital letter.  We used \\"What is\\" instead of \\"whats\\" to sound more proper. ", "New York is a proper noun, so it needs a capital letter for both words."]}'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 101, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 101}], 'prompt_token_count': 368, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 368}], 'total_token_count': 469, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-705d7765-12a1-463d-82be-42bff5a9efac', 'author': 'agent_grammar', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_con

Conversation Flow:

The session contains multiple interactions where the user asks questions with intentional misspellings, and the agent corrects them. Here are the main interactions:

Response Format: Each agent response is in JSON format containing:

- original_query: The user's input
- corrected_text: The corrected version
- errors: List of error types found
- explanations: Detailed explanations of the corrections
- Technical Details: Each interaction has a unique invocation_id
- Events are timestamped
- The session maintains state and tracks all events


To summarize, this agent:

1. Accepts user input
2. Identifies grammatical and spelling errors
3. Provides corrected versions
4. Explains the corrections
5. Maintains a conversation history
6. The agent is particularly focused on:
7. Spelling corrections (e.g., "ho" → "how", "whut" → "what")
8. Sentence structure (e.g., adding missing "be")
9. Providing educational explanations for each correction

# Hosting Agent on Agent engine

In [60]:
model = "gemini-2.0-flash-001"

from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]
generate_content_config = types.GenerateContentConfig(
    safety_settings=safety_settings,
    temperature=0.28,
    max_output_tokens=1000,
    top_p=0.95,
)

import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

STAGING_BUCKET = GCS_BUCKET_URI

PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}
LOCATION = "us-central1" #@param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [61]:
from google.adk.agents import Agent

agent = Agent(
    model=model,                                      # Required.
    name='currency_exchange_agent',                   # Required.
    generate_content_config=generate_content_config,  # Optional.
)
app = AdkApp(agent=agent)
for event in app.stream_query(
    user_id="ysian",  # Required
    message="What is the exchange rate from US dollars to Swedish currency?",
):
    print(event)

{'content': {'parts': [{'text': "Please specify which Swedish currency you're asking about. Sweden uses the Swedish Krona (SEK). Once you confirm that's what you're looking for, I can provide the current exchange rate.\n"}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 42, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 42}], 'prompt_token_count': 28, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 28}], 'total_token_count': 70, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-3c81f408-5e87-40d5-a110-7c142baacf8e', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'AolTACAo', 'timestamp': 1750686678.472449}


## Enable Tracability

In [62]:
app = AdkApp(
    agent=agent,
    enable_tracing=True, # <- new
)

for event in app.stream_query(
    user_id="saurabh",
    message="What is the exchange rate from US dollars to Swedish currency on 2025-04-03?",
):
    print(event)

{'content': {'parts': [{'text': "I can't provide you with a specific exchange rate for USD to SEK on April 3, 2025. Exchange rates fluctuate constantly based on market conditions. You'll need to check a reliable source closer to that date to get an accurate rate.\n\nHere's where you can find exchange rates:\n\n*   **Financial websites:** Reputable financial websites (like Google Finance, Yahoo Finance, Bloomberg, etc.) will provide current and historical exchange rates.\n*   **Currency converters:** Many online currency converters are available, but make sure they use a reliable data source.\n*   **Banks and financial institutions:** Your bank or a currency exchange service will give you the current exchange rate they are offering."}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 147, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 147}], 'prompt_token_count': 40, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT

## Agent Engine hosting

In [63]:
remote_app = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],
    display_name="exchange_rate_agent",
    description="Agent Engine that uses ADK",        
)

Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.98.0', 'cloudpickle': '3.0.0'}


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.98.0', 'cloudpickle': '3.0.0'}


The following requirements are missing: {'pydantic', 'cloudpickle'}


The following requirements are appended: {'cloudpickle==3.0.0', 'pydantic==2.11.7'}


INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.0.0', 'pydantic==2.11.7'}


...

INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/255766800726/locations/us-central1/reasoningEngines/9072334323570442240


To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/255766800726/locations/us-central1/reasoningEngines/9072334323570442240')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/255766800726/locations/us-central1/reasoningEngines/9072334323570442240')


In [64]:
import pprint

pprint.pprint(remote_app.operation_schemas())

[{'api_mode': '',
  'description': 'Get a session for the given user.',
  'name': 'get_session',
  'parameters': {'properties': {'session_id': {'type': 'string'},
                                'user_id': {'type': 'string'}},
                 'required': ['user_id', 'session_id'],
                 'type': 'object'}},
 {'api_mode': '',
  'description': 'List sessions for the given user.',
  'name': 'list_sessions',
  'parameters': {'properties': {'user_id': {'type': 'string'}},
                 'required': ['user_id'],
                 'type': 'object'}},
 {'api_mode': '',
  'description': 'Creates a new session.',
  'name': 'create_session',
  'parameters': {'properties': {'session_id': {'nullable': True,
                                               'type': 'string'},
                                'state': {'nullable': True, 'type': 'object'},
                                'user_id': {'type': 'string'}},
                 'required': ['user_id'],
                 'type': 'object'

# Agent Features

## Basic Agent

In [65]:
basic_agent = Agent(
    name='basic_agent',
    description='A helpful assistant for user.',
    model='gemini-2.0-flash-001',
    instruction="Answer user's question with your best knowledge.",
)

In [69]:
runner = create_runner(basic_agent)
session =  run_session(content_text('Hello!'), runner=runner)
session =  run_session(content_text('Tell me a joke under 30 words?'), runner=runner, session=session)

In [71]:
session

<coroutine object run_session at 0x7faad7d8f140>

In [84]:
call = await caller_factory(root_agent=basic_agent)
pprint_events(call('Tell me a joke under 30 words'))

>>> Final Response (0.069 ms):
Why don't scientists trust atoms?

Because they make up everything!

------------------------------
Total elapsed time: 0.201 ms


### Agent declaration

In [85]:
sub_agent_1 = Agent(
    name='sub_agent_1',
    description='No.1 sub agent.',
    model='gemini-2.0-flash-001',
    instruction="JUST SAY 1.",
)

sub_agent_2 = Agent(
    name='sub_agent_2',
    description='No.2 sub agent.',
    model='gemini-2.0-flash-001',
    instruction="JUST SAY 2.",
)

## Sequential Agent

In [86]:
sequential_agent = SequentialAgent(
    name='sequential_agent',
    sub_agents=[sub_agent_1, sub_agent_2],
)

In [87]:
call = await caller_factory(root_agent=sequential_agent)
pprint_events(call('hi'))

>>> Final Response (0.059 ms):
1.

------------------------------
>>> Final Response (0.204 ms):
2.

------------------------------
Total elapsed time: 0.229 ms


In [88]:
# Expect output:
"""
Author: child_1_agent
Content Text: 1.

...

Author: child_2_agent
Content Text: 2.
"""

'\nAuthor: child_1_agent\nContent Text: 1.\n\n...\n\nAuthor: child_2_agent\nContent Text: 2.\n'

## Parallel Agent

In [89]:
from typing import Optional, Any

APP_NAME = 'test_app'
USER_ID = 'test_user'

def create_runner(agent: BaseAgent):
  return InMemoryRunner(agent, app_name=APP_NAME)

def _content_to_text(content: types.Content | None) -> str:
  if not content or not content.parts:
    return ''
  return ''.join([p.text or '' for p in content.parts])

def print_event(event: Event):
  print(f'Author: {event.author}')
  print(f'Content Text: {_content_to_text(event.content)}')
  print(f'Event: {event.model_dump(exclude_none=True, exclude_defaults=True)}')

async def run_session(new_message: types.Content, *, runner: Runner, session: Optional[Session] = None, state: Optional[dict[str, Any]] = None) -> Session:
  if session is None:
    session = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, state=state)

  print(f'User: {_content_to_text(new_message)}')
  print('----------------------------------')
  async for e in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=new_message):
    print_event(e)
  print('----------------------------------')

  session = runner.session_service.get_session(app_name=session.app_name, user_id=session.user_id, session_id=session.id)
  return session

def content_text(msg: str) -> types.Content:
  return types.UserContent(parts=[types.Part(text=msg)])

In [90]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  return random.randint(1, sides)


single_agent_1 = Agent(
    name='async_agent_1',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 8 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
single_agent_2 = Agent(
    name='async_agent_2',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 10 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
single_agent_3 = Agent(
    name='async_agent_3',
    model='gemini-2.0-flash-001',
    instruction='Just roll a dice of 15 sides when being asked to roll a die.',
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=False,
    tools=[roll_die],
)
parallel_agent = ParallelAgent(
    name='parallel_agent',
    sub_agents=[
        single_agent_1,
        single_agent_2,
        single_agent_3,
    ],
)


In [92]:
call = await caller_factory(root_agent=parallel_agent)
pprint_events(call('Hello'))

ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7faaf252f100> at 0x7faad4410e80> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7faaf252f100> at 0x7faad7ddb400> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (

>>> Final Response (0.066 ms):
Hello! How can I help you today?

------------------------------
>>> Final Response (0.182 ms):
Hello! How can I assist you today?

------------------------------
>>> Final Response (0.226 ms):
Hello! How can I assist you today?

------------------------------
Total elapsed time: 0.25 ms


In [93]:
call = await caller_factory(root_agent=parallel_agent)
pprint_events(call('Roll a dice of 100 sides and tell me what you got'))

ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7faaf252f100> at 0x7faad442a040> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x7faaf252f100> at 0x7faad443ee40> was created in a different Context
ERROR:opentelemetry.context:Failed to detach context
Traceback (

+++ Function Calls:
Function Name: roll_die, Args: {'sides': 100}
--- Function Responses:
Function Name: roll_die
Function Results: [55]
+++ Function Calls:
Function Name: roll_die, Args: {'sides': 100}
--- Function Responses:
Function Name: roll_die
Function Results: [6]
+++ Function Calls:
Function Name: roll_die, Args: {'sides': 100}
--- Function Responses:
Function Name: roll_die
Function Results: [1]
>>> Final Response (0.514 ms):
I rolled a 100-sided die and got 55.

------------------------------
>>> Final Response (0.568 ms):
I rolled a 100-sided die and got 6.

------------------------------
>>> Final Response (0.638 ms):
I rolled a 100-sided die and got a 1.

------------------------------
Total elapsed time: 0.665 ms


### Loop agent

In [94]:
from google.adk.agents import LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.agent_tool import AgentTool
from google.adk.agents.loop_agent import LoopAgent

from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
print(json_schema)

MODEL = "gemini-2.0-flash-001"

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
    # allow_transfer=False,
)

def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result

simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    # children=[agent_math],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

from google.adk.tools.tool_context import ToolContext

def exit_loop(tool_context: ToolContext):
  tool_context.actions.escalate = True

agent_checker = Agent(
    model=MODEL,
    name="agent_checker",
    description="This agent checks if the kid's query is fulfilled. It escalates if both grammar is fixed and the math is calculated",
    instruction="""
    Analyze the chat log between the user (kids) and other agents.

    We are expecting two outcomes from the conversation.
    1. agent_grammar helps to fix the grammar of kid's question.
    2. agent_math_advanced tries to solve the math step by step.

    Decide the action and respond as follows:

    1. If both agent_grammar and agent_math_advanced have done their task, summarize the answer in a friendly way to the kid. Then call exit_loop.

    2. If there is no pending math problem to solve, friendly tell the kid you help fix grammar and solve math, but does not know anything else. Politely ask for math question. Then call exit_loop.

    3. If agent_grammar has fixed the grammar but agent_math_advanced has not generated a final answer with single number, extract corrected_text from most recent JSON response and empahsize that is the math question to solve.

    3. Otherwise, say "Thanks for the question! I'll let grammar and math agent to help you!". Do not call any tools in this case.

    """,
    tools=[exit_loop]
)

agent_extractor = Agent(
    model=MODEL,
    name="agent_extractor",
    description="Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.",
    instruction="""
    Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    input_schema=OutputSchema,
)

agent_teaching_assistant_loop = LoopAgent(
    name="agent_teaching_assistant_loop",
    description="This agent acts as a friendly teaching assistant, checking the grammar of kids' questions, performing math calculations using corrected or original text (if grammatically correct), and providing results or grammar feedback in a friendly tone.",
    sub_agents=[agent_checker, agent_grammar, agent_extractor, agent_math_advanced],
)

root_agent = agent_teaching_assistant_loop

{'properties': {'original_query': {'description': 'The original text from user.', 'title': 'Original Query', 'type': 'string'}, 'corrected_text': {'description': 'The corrected text.', 'title': 'Corrected Text', 'type': 'string'}, 'errors': {'description': 'An array of descriptions of each error.', 'items': {'type': 'string'}, 'title': 'Errors', 'type': 'array'}, 'explanations': {'description': 'An array of explanations for each correction.', 'items': {'type': 'string'}, 'title': 'Explanations', 'type': 'array'}}, 'required': ['original_query', 'corrected_text', 'errors', 'explanations'], 'title': 'OutputSchema', 'type': 'object'}


In [95]:
call = await caller_factory(root_agent=agent_teaching_assistant_loop)
pprint_events(call('ok tell me how i can count water.'))

ERROR:opentelemetry.context:Failed to detach context
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/trace/__init__.py", line 587, in use_span
  File "/opt/conda/lib/python3.10/site-packages/opentelemetry/sdk/trace/__init__.py", line 1105, in start_as_current_span
    yield span
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/trace/__init__.py", line 452, in start_as_current_span
  File "/opt/conda/lib/python3.10/site-packages/google/adk/agents/base_agent.py", line 148, in run_async
    yield event
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/__init__.py", line 155, in detach
  File "/home/jupyter/.local/lib/python3.10/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
ValueError: <Token var=<ContextVar n

>>> Final Response (0.05 ms):
Thanks for the question! I'll let grammar and math agent to help you!

------------------------------
>>> Final Response (0.17 ms):
{
  "original_query": "Roll a dice of 100 sides and tell me what you got",
  "corrected_text": "Roll a 100-sided die and tell me what you get.",
  "errors": [
    "Incorrect article usage ('a dice' should be 'a die').",
    "Incorrect word choice ('sides' implies multiple, but we're talking about one die with many sides).",
    "Incorrect tense ('got' should be 'get' for a request)."
  ],
  "explanations": [
    "When we talk about just one of those number cubes, it's called a 'die' (like saying 'one cookie'). If you have more than one, then they are called 'dice' (like saying 'many cookies').",
    "It's better to say '100-sided die' because you only have one die, but it has 100 sides!",
    "When you ask someone to tell you what they will get in the future, it's best to use the word 'get'. 'Got' means you already have it!"
 

In [ ]:
runner = create_runner(agent_teaching_assistant_loop)
session = await run_session(content_text('Hello!'), runner=runner)
session = await run_session(content_text('ok tell me how i can count water.'), runner=runner)